In [20]:
import pandas as pd
import numpy as np
import vocab_mappings as vocab

In [19]:
%cd ..

/Users/yc3972/Desktop/DBMI/Courses/GD1_Fall/G4003 Symbolic Methods/Project


In [132]:
import off_label

## Read in Sample Patient Data

In [22]:
def returning_lists_alt(drug,i):
    patients_given = off_label.patient_list(drug)
    query_patient = patients_given.subject_id[i] 
    query_visit = patients_given.hadm_id[i]
    print("Searching for Patient "+ str(query_patient) + " who has been given " + str(drug) + " , for visit number " + str(query_visit) + ".")
    diagnoses = off_label.diagnosis_list(query_patient, query_visit)
    drugs = off_label.drugs_list(query_patient, query_visit)
    
    return diagnoses, drugs

In [136]:
diagnoses0,drugs0 = off_label.returning_lists('Gabapentin',0) 
diagnoses390,drugs390 = off_label.returning_lists('Gabapentin',390) 
diagnoses40007,drugs40007 = off_label.returning_lists('Gabapentin',40007) 

Searching for Patient 17868682 who has been given Gabapentin , for visit number 25218370.
Searching for Patient 11443746 who has been given Gabapentin , for visit number 27515077.
Searching for Patient 15214984 who has been given Gabapentin , for visit number 24427705.


In [134]:
import imp
imp.reload(off_label)
#%pwd

<ipython-input-134-41b5a42cd49e>:2: DtypeWarning: Columns (7) have mixed types.Specify dtype option on import or set low_memory=False.
  imp.reload(off_label)


<module 'off_label' from '/Users/yc3972/Desktop/DBMI/Courses/GD1_Fall/G4003 Symbolic Methods/Project/off_label/off_label.py'>

In [34]:
std_diag_dict0 = vocab.diag_ICD_SNOMED(diagnoses0)
std_diag_dict390 = vocab.diag_ICD_SNOMED(diagnoses390) # Found random patients whose diagnoses would match indications
std_diag_dict40007 = vocab.diag_ICD_SNOMED(diagnoses40007)

In [8]:
diag_MeSH_dict0 = vocab.diag_ICD_MeSH(diagnoses0)
diag_MeSH_dict390 = vocab.diag_ICD_MeSH(diagnoses390)


In [42]:
diag_MeSH_dict40007 = vocab.diag_ICD_MeSH(diagnoses40007)

In [43]:
diag_MeSH_dict40007

{'ICD9 280.9': {'SNOMED': [436659],
  'MeSH': [45617133],
  'MeSH_concept': ['Anemia, Iron-Deficiency']},
 'ICD9 333.94': {'SNOMED': [73754],
  'MeSH': [45610854],
  'MeSH_concept': ['Restless Legs Syndrome']},
 'ICD9 288.60': {'SNOMED': [438398],
  'MeSH': [45619141],
  'MeSH_concept': ['Leukocytosis']},
 'ICD9 414.01': {'SNOMED': [317576], 'MeSH': [], 'MeSH_concept': []},
 'ICD9 458.0': {'SNOMED': [319041],
  'MeSH': [45614300],
  'MeSH_concept': ['Hypotension, Orthostatic']},
 'ICD9 564.00': {'SNOMED': [75860],
  'MeSH': [45616518],
  'MeSH_concept': ['Constipation']},
 'ICD9 787.91': {'SNOMED': [196523],
  'MeSH': [45618962],
  'MeSH_concept': ['Diarrhea']}}

# Patient Data to Indication Data Mappings

## Patient Data --> DrugCentral
TODO: first few cells of formatting from gb_DrugCentralRaw --> gb_DrugCentral: KS to in DrugInd_Extraction code to correct for SNOMED concept code vs ID

In [126]:
%cd ~/Desktop/DBMI/Courses/GD1_Fall/G4003 Symbolic Methods/Project/off_label/full_OMOP

/Users/yc3972/Desktop/DBMI/Courses/GD1_Fall/G4003 Symbolic Methods/Project/off_label/full_OMOP


In [127]:
# Change directory to where OMOP Vocabulary download is saved
# Load all concepts
concept = pd.read_csv('CONCEPT.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id','concept_name',
                              'domain_id','vocabulary_id','concept_class_id','standard_concept',
                              'concept_code','valid_start_date','valid_end_date','invalid_reason'],index_col=False)
# Load all concept relationships
concept_relationship = pd.read_csv('CONCEPT_RELATIONSHIP.csv',delimiter='\t',skiprows=1,
                      names = ['concept_id_1','concept_id_2',
                              'relationship_id',
                               'valid_start_date','valid_end_date','invalid_reason'],
                           index_col=False)

/Users/yc3972/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (5,6,9) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [128]:
%cd ..

/Users/yc3972/Desktop/DBMI/Courses/GD1_Fall/G4003 Symbolic Methods/Project/off_label


In [85]:
# Load DrugCentral data
gb_DrugCentral_raw = pd.read_csv('Gabapentin_indications_drugcentral.csv', sep = ',', header =0)
gb_DrugCentral_raw['snomed_conceptid'] = gb_DrugCentral_raw['snomed_conceptid'].fillna(0).astype(np.int64)

# Join nonstandard conceptIDs with OMOP concept table to get data on the concept name, vocabualry source, standard flag, etc. 
merge_DC = pd.merge(gb_DrugCentral_raw, concept, 
                    how="left", left_on='snomed_conceptid', right_on='concept_code')
merge_DC = merge_DC.loc[merge_DC['vocabulary_id']!='Nebraska Lexicon'].reset_index()

gb_DrugCentral = merge_DC[['struct_id','omop_concept_id','relationship_name','omop_concept_name','umls_cui','umls_semantic_type','snomed_conceptid','snomed_full_name','drug_name','concept_id','concept_class_id','standard_concept']]
gb_DrugCentral =  gb_DrugCentral.rename(columns = {'snomed_conceptid':'snomed_concept_code','concept_id':'snomed_concept_id'})
gb_DrugCentral['snomed_concept_id'] = gb_DrugCentral['snomed_concept_id'].fillna(0).astype(np.int64)

# Convert all SNOMED ID's from DrugCentral to lists for each classification of indication, off-label use, contraindication
DC_SNOMED_ind = (gb_DrugCentral.loc[gb_DrugCentral['relationship_name']=='indication'])['snomed_concept_id'].values.tolist()
DC_SNOMED_off = (gb_DrugCentral.loc[gb_DrugCentral['relationship_name']=='off-label use'])['snomed_concept_id'].values.tolist()
DC_SNOMED_contr = (gb_DrugCentral.loc[gb_DrugCentral['relationship_name']=='contraindication'])['snomed_concept_id'].values.tolist()


In [93]:
DC_SNOMED_ind

[4071164, 73754, 0]

In [166]:
### Get All related SNOMED codes for given SNOMED code, HORIZONTALLY
### For mapping, refer to vocab_mapping.py, get_nonStandard_SNOMED(j)
### We specifically want to get all the SNOMED codes that map to out standard, for efficient comparison with indication
def all_related(std_dict):
    related = {}
    for icd in std_dict.keys():
        std_codes = std_dict[icd]
        for j in std_codes:
            all_related = vocab.get_nonStandard_SNOMED(j)
            all_related = all_related.loc[all_related.vocabulary_id == 'SNOMED','concept_id_2'].values.tolist()
            if icd not in related:
                related[icd] = all_related
            else:
                related[icd].extend(all_related)
    return related

In [167]:
all_snomed_0 = all_related(std_diag_dict0)

In [170]:
all_snomed_390 = all_related(std_diag_dict390)
all_snomed_40007 = all_related(std_diag_dict40007)

In [171]:
# TO DO: for now just prints, create dictionary output, or whatever makes the most sense 
# TO DO: count matches
# get_DrugCentral_matches HORIZONTALLY
# Input dictionary of ICD codes and SNOMED codes
# Output which SNOMED codes map to DrugCentral Indication data
def get_DC_matches(diag_dict):
    for icd in diag_dict.keys():
        std_codes = diag_dict[icd]
        for j in std_codes:
            if j in DC_SNOMED_ind: # an indication
                print('indication:')
                print(j)
            elif j in DC_SNOMED_off: # off label use
                print('off-label:')
                print(j)
            elif j in DC_SNOMED_contr: # contraindication
                print('contraindication:')
                print(j)
            else:
                pass 

In [172]:
get_DC_matches(all_snomed_0)

contraindication:
440383


In [173]:
get_DC_matches(all_snomed_390)

contraindication:
4273391
off-label:
436962


In [174]:
get_DC_matches(all_snomed_40007)

indication:
73754


## 3b. Patient Data --> AACT

In [33]:
# Extract from AACT database all mesh terms for completed/ongoing clinical trials where the intervention is Gabapentin
# CSV exported from: OneDrive%20-%20cumc.columbia.edu/Symbolic_Project/DrugInd_Extraction.ipynb

gb_AACT = pd.read_csv('Gabapentin_treat_ClinicalTrials.csv',skiprows=1,
                      names = ['nct_id','drug_mesh','cond_mesh','mesh_type'],index_col=False)
gb_AACT

,nct_id,drug_mesh,cond_mesh,mesh_type
0,NCT00619983,Gabapentin,Neuralgia,mesh-list
1,NCT00619983,Gabapentin,Diabetic Neuropathies,mesh-list
2,NCT00619983,Gabapentin,Back Pain,mesh-list
3,NCT00619983,Gabapentin,Low Back Pain,mesh-list
4,NCT00619983,Gabapentin,Pain,mesh-ancestor
...,...,...,...,...
2078,NCT00666770,Gabapentin,"Sleep Disorders, Intrinsic",mesh-ancestor
2079,NCT00666770,Gabapentin,Dyssomnias,mesh-ancestor
2080,NCT00666770,Gabapentin,Sleep Wake Disorders,mesh-ancestor
2081,NCT00666770,Gabapentin,Nervous System Diseases,mesh-ancestor


In [48]:
def get_AACT_matches(diag_MeSH_dict):
    
    for ICD in diag_MeSH_dict:
        concept = diag_MeSH_dict[ICD]['MeSH_concept'][0]

        if (gb_AACT.loc[gb_AACT['cond_mesh'] == concept]).empty == False:
            display(gb_AACT.loc[gb_AACT['cond_mesh'] == concept])

    return

In [49]:
get_AACT_matches(diag_MeSH_dict0)

,nct_id,drug_mesh,cond_mesh,mesh_type
925,NCT01263132,Gabapentin,Polyneuropathies,mesh-list


,nct_id,drug_mesh,cond_mesh,mesh_type
1558,NCT02987985,Gabapentin,Depressive Disorder,mesh-list


,nct_id,drug_mesh,cond_mesh,mesh_type
524,NCT03476538,Gabapentin,Obesity,mesh-ancestor
2013,NCT00886236,Gabapentin,Obesity,mesh-ancestor


,nct_id,drug_mesh,cond_mesh,mesh_type
1553,NCT02987985,Gabapentin,"Sleep Apnea, Obstructive",mesh-list


,nct_id,drug_mesh,cond_mesh,mesh_type
322,NCT04700592,Gabapentin,"Hernia, Inguinal",mesh-list
744,NCT02419443,Gabapentin,"Hernia, Inguinal",mesh-list
1114,NCT00735124,Gabapentin,"Hernia, Inguinal",mesh-list


,nct_id,drug_mesh,cond_mesh,mesh_type
621,NCT03088306,Gabapentin,Spinal Stenosis,mesh-list
1951,NCT01127100,Gabapentin,Spinal Stenosis,mesh-list


In [50]:
get_AACT_matches(diag_MeSH_dict390)

,nct_id,drug_mesh,cond_mesh,mesh_type
524,NCT03476538,Gabapentin,Obesity,mesh-ancestor
2013,NCT00886236,Gabapentin,Obesity,mesh-ancestor


,nct_id,drug_mesh,cond_mesh,mesh_type
623,NCT03088306,Gabapentin,Bone Diseases,mesh-ancestor
802,NCT02120703,Gabapentin,Bone Diseases,mesh-ancestor
1031,NCT01014520,Gabapentin,Bone Diseases,mesh-ancestor
1392,NCT03867240,Gabapentin,Bone Diseases,mesh-ancestor
1959,NCT01127100,Gabapentin,Bone Diseases,mesh-ancestor
2049,NCT00684112,Gabapentin,Bone Diseases,mesh-ancestor


,nct_id,drug_mesh,cond_mesh,mesh_type
2,NCT00619983,Gabapentin,Back Pain,mesh-list
240,NCT00108550,Gabapentin,Back Pain,mesh-list


## 3c. MIMIC NDC codes to RxNorm Ingredient
MIMIC drug codes are documented as NDC codes. We convert these NDC codes to RxNorm ingredient, then convert them to NDFRT codes that contain indication information from OMOP.

For Diagnosis information, SNOMED terms must be converted to NDFRT terms to find matching.

In [141]:
import concept_mapping

#### Part I : Converting Drug codes to RxNorm, then to NDFRT codes

In [177]:
# Drug dictionary that stores the standard RxNorm codes for the given drugs
# Used if we were to traverse across drugs (to find related other ndc codes)
# NDC codes --> RxNorm standard 
def NDC_RxNorm(drugslist):
    std_drug_dict = {}
    for i in range(drugslist.shape[0]):
        drug = drugslist.loc[i,'drug']
        ndc = drugslist.loc[i, 'ndc']

        # ndc = 0 means there is no ndc code
        if ndc == '0':
            continue
        ndc_id = concept.loc[concept.concept_code == ndc, 'concept_id']

        if len(ndc_id) == 0: # there is no matching standard
            continue
        ndc_id = ndc_id.item()
        standard_id = concept_mapping.maps_to(concept_relationship, ndc_id)
        ingredient = concept_mapping.find_ingredient(drug)
        # if rxnorm ingredient cannot be found directly, manually find via relationships
        if len(ingredient) == 0 or len(ingredient) > 1: 
            ingredient = [concept_mapping.find_ingredient2(standard_id)]

        if drug not in std_drug_dict: 
            std_drug_dict[drug] = ingredient
    return std_drug_dict

In [181]:
rxnorm0 = NDC_RxNorm(drugs0)

In [179]:
rxnorm390 = NDC_RxNorm(drugs390)
rxnorm40007 = NDC_RxNorm(drugs40007)

In [195]:
rxnorm0

{'Acetaminophen': [1125315],
 'Docusate Sodium': [19126912],
 'Senna': [938268],
 'OxyCODONE (Immediate Release)': [1124957],
 'Gabapentin': [797399],
 'Tamsulosin': [924566],
 'Heparin': [1367571],
 'BuPROPion XL (Once Daily)': [40221871],
 'Warfarin': [1310149],
 'Ketorolac': [1136980]}

In [211]:
# Finding the NDFRT equivalent of that rxnorm ingredient, necessary for the treat relationships
# Find mapping code in concept_mapping.py
# Takes in rxnorm ingredient list, returns dictionary of ndfrt equivalents

def all_ndfrt(rxnorm_list):
    ndfrt_list = {}
    for drug in rxnorm_list.keys():
        rxnorm = rxnorm_list[drug][0]
        ndfrt = concept_mapping.find_NDFRT(rxnorm)
        if ndfrt == None:
            print(drug, "doesn't have ndfrt equivalent")
            continue
        if drug not in ndfrt_list:
            ndfrt_list[drug] = ndfrt
    return ndfrt_list

In [217]:
ndfrt0 = all_ndfrt(rxnorm0)
ndfrt390 = all_ndfrt(rxnorm390)
ndfrt40007 = all_ndfrt(rxnorm40007)

BuPROPion XL (Once Daily) doesn't have ndfrt equivalent
Multivitamins doesn't have ndfrt equivalent
Topiramate (Topamax) doesn't have ndfrt equivalent
Hydrocortisone Cream 2.5% doesn't have ndfrt equivalent
PNEUMOcoccal 23-valent polysaccharide vaccine doesn't have ndfrt equivalent
Polyethylene Glycol doesn't have ndfrt equivalent


In [228]:
ndfrt0

{'Acetaminophen': [4287137, 4350076],
 'Docusate Sodium': [4292316, 40182085],
 'Senna': [4272742, 4293852, 40180868],
 'OxyCODONE (Immediate Release)': [4293104, 4325316],
 'Gabapentin': [4272456, 4351183],
 'Tamsulosin': [4274667, 4325341],
 'Heparin': [4272103, 4348060],
 'Warfarin': [4293218, 4325514],
 'Ketorolac': [4349348, 40180393]}

In [218]:
# Flatten ndfrt dictionary to all the ndfrt codes for the drugs given to this patient 
def flattened_ndfrt(ndfrt_dict):
    return [item for sublist in ndfrt_dict.values() for item in sublist]

In [219]:
list0 = flattened_ndfrt(ndfrt0)
list390 = flattened_ndfrt(ndfrt390)
list40007 = flattened_ndfrt(ndfrt40007)

#### Part II: Find NDFRT codes for diagnosis

In [251]:
# Utilize the same nonstandard mapping, but we want NDFRT codes instead of SNOMED
def all_related_NDFRT(std_dict):
    related = {}
    for icd in std_dict.keys():
        std_codes = std_dict[icd]
        for j in std_codes:
            all_related = vocab.get_nonStandard_SNOMED(j)
            all_related = all_related.loc[all_related.vocabulary_id == 'NDFRT','concept_id_2'].values.tolist()
            if icd not in related:
                related[icd] = all_related
            else:
                related[icd].extend(all_related)
    return related

In [252]:
all_ndfrt_0 = all_related_NDFRT(std_diag_dict0)
all_ndfrt_390 = all_related_NDFRT(std_diag_dict390)
all_ndfrt_40007 = all_related_NDFRT(std_diag_dict40007)

In [255]:
concept.loc[concept.concept_id == 201231]

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
6598144,201231,Malignant neoplasm of connective and soft tiss...,Condition,SNOMED,Clinical Finding,S,188019007,20020131,20991231,NaN


In [256]:
diagnoses0

,index,subject_id,hadm_id,seq_num,icd_code,icd_version,long_title
0,5170387,17868682,25218370,1,C495,10,Malignant neoplasm of connective and soft tiss...
1,4434573,17868682,25218370,2,G629,10,"Polyneuropathy, unspecified"
2,4316971,17868682,25218370,3,I482,10,Chronic atrial fibrillation
3,4084054,17868682,25218370,4,I340,10,Nonrheumatic mitral (valve) insufficiency
4,3547038,17868682,25218370,5,F329,10,"Major depressive disorder, single episode, uns..."
5,3209088,17868682,25218370,6,E669,10,"Obesity, unspecified"
6,3804012,17868682,25218370,7,G4733,10,Obstructive sleep apnea (adult) (pediatric)
7,4563330,17868682,25218370,8,K4090,10,"Unilateral inguinal hernia, without obstructio..."
8,5213438,17868682,25218370,9,N503,10,Cyst of epididymis
9,4987217,17868682,25218370,10,R590,10,Localized enlarged lymph nodes


In [183]:
#concept.loc[concept.concept_id == 4272456]
# Takes a list of NDFRT drug codes and returns all the NDFRT diagnosis codes that may be treated by the drug
def omop_indications(ndfrt_list):
    indications = []
    for i in range(len(ndfrt_list)):
        concepts = concept_relationship.loc[(concept_relationship.concept_id_1 == ndfrt_list[i]) & 
                         (concept_relationship.relationship_id == 'May treat'), 'concept_id_2'].values.tolist()
        indications.extend(concepts)
    return indications

In [220]:
omop_list0 = omop_indications(list0)
omop_list390 = omop_indications(list390)
omop_list40007 = omop_indications(list40007)

In [269]:
# Now, we try to map these NDFRT indications back to SNOMED so that we can compare with our diagnoses
def ndfrt_SNOMED(ndfrt_list):
    snomed_list = []
    for ndfrt in ndfrt_list:
        snomed = concept_relationship.loc[(concept_relationship.concept_id_1 == ndfrt)&
                        (concept_relationship.relationship_id == 'Ind/CI - SNOMED'), 'concept_id_2'].values.tolist()
        snomed_list.extend(snomed)
    return snomed_list

In [271]:
snomed_indications0 = ndfrt_SNOMED(omop_list0)
snomed_indications390 = ndfrt_SNOMED(omop_list390)
snomed_indications40007 = ndfrt_SNOMED(omop_list40007)

In [221]:
# Asking if there are matches in S
def find_omop_matches(diag_list, omop_list):
    for icd in diag_list.keys():
        std_codes = diag_list[icd]
        for j in std_codes:
            if j in omop_list:
                print("OMOP reported indication relationship for ", j)

In [275]:
find_omop_matches(all_snomed_0, snomed_indications0)
find_omop_matches(all_snomed_390, snomed_indications390)
find_omop_matches(all_snomed_40007, snomed_indications40007)
## Sadly, OMOP doesn't match much with horizontal codes

OMOP reported indication relationship for  4101454
OMOP reported indication relationship for  4101454
OMOP reported indication relationship for  75860
